In [ ]:
#Warning : this code is not orgnaized yet
import requests
import base64, json
import spotipy as sp
import pandas as pd
import numpy as np

In [ ]:
#curl -X "POST" -H "Authorization: Basic ZjM4ZjAw...WY0MzE=" -d grant_type=client_credentials https://accounts.spotify.com/api/token

authUrl="https://accounts.spotify.com/api/token"

authHeader= {}

authData ={}

clientID =""

clientSecret =""

message = f'{clientID}:{clientSecret}'
message_bytes = message.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

print(base64_message)

In [ ]:

authData['grant_type']='client_credentials'

authHeader['Authorization']= "Basic "+base64_message

res =requests.post(authUrl, headers=authHeader, data= authData)

print(res)

In [ ]:
responseObject = res.json()

acssesToken = responseObject['access_token']

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth=acssesToken)

In [ ]:
# method to get the play list object (dont use this)

def getplaylist(token,id_):
    playlistendpoint = f'https://api.spotify.com/v1/playlists/{id_}'
    
    GetHeader = { 
    "Authorization":"Bearer "+token
    }
    
    res = requests.get(playlistendpoint, headers=GetHeader)
    
    playlistObject = res.json()
    
    return playlistObject
    

In [ ]:
# this method is not working somehow 
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id= playlist_id)
    tracks = results['tracks']
    while tracks['next']:
        results = sp.next(results)
        tracks.extend(results['tracks'])
    return tracks

This code brings only 100 song from each playlist..

In [ ]:

playlist__ = ['2UZk7JjJnbTut1w8fqs3JL?si=291615e5d4564352','4bjo8xl4C0qTLIvv7KL850?si=0a3be6aeabad4cea','1oMdoSZKXxt3Aq0m08FoFp?si=57e044891e014bc4',
             '12DQo1bBqzbZPyR52v21um?si=a84c983ac5654732','60BPbvNDih1rjejhylAtbP?si=b351f288e9c9433c','2GbllvHLyTUGzOuSVWKGhV?si=c184232ecf934fbf',
             '37i9dQZF1DX5cO1uP1XC1g?si=4171cadd521f4717','37i9dQZF1DXcJUwMZo8Ss1?si=0d8f87de4f0b497d','37i9dQZF1DX51I48C1U653?si=432e29c977d34e0b',
             '37i9dQZF1DWUa8ZRTfalHk?si=0e42f6e9ef814053','37i9dQZF1DX3WvGXE8FqYX?si=4d51052b7e244aee','37i9dQZF1DWWEcRhUVtL8n?si=6b1dd1d1c5c24db1',
             '37i9dQZF1DXbIGqYf7WDxP?si=8738ac36e026455b','37i9dQZF1DWYp3yzk1civi?si=9eb52966ad8c4ce4','37i9dQZF1DWYs2pvwxWA7l?si=5c8a122686494a3c',
             '37i9dQZF1DXcTieYAg7jq1?si=f03ebf18a6db421a','37i9dQZF1DX3u9TSHqpdJC?si=af5ed9afa5184668','37i9dQZF1DWYkaDif7Ztbp?si=79a259d6a7bb41d2',
             '37i9dQZF1DX2L0iB23Enbq?si=74a69b8f24204ba0','37i9dQZF1DWYs83FtTMQFw?si=425ffdded1fa4ff4','37i9dQZF1DXaPCIWxzZwR1?si=86106a41f8754485',
             '37i9dQZF1DWX9u2doQ8Q2L?si=8173cc8c0541470d','37i9dQZF1DWY8U6Zq7nvbE?si=5c8bd3069378413a','37i9dQZF1DWY8U6Zq7nvbE?si=05ef3bd3c40d4e8b',
             '37i9dQZF1DX4pUKG1kS0Ac?si=468936cf07854fb1','2uSJiLBLIj0Q9fYnbImWQt?si=33acce5196e441d1','3W1EC3RNOdJxep69RJD0RS?si=1cdd67d4ccba4819',
             '37i9dQZF1DX6aTaZa0K6VA?si=e924f025d7e743fe','6mtYuOxzl58vSGnEDtZ9uB?si=4da0d636149c454a','37i9dQZF1DX8ttEdg9VJHO?si=f1848b01b4c54832']
id_list = []
songlist_ = []
artistlist_=[]
#print(json.dumps(tracks,indent=2))
dict_ = {}

        
for i in range(len(playlist__)) :
    
    tracks = getplaylist(acssesToken,playlist__[i])
    
    for x in tracks['tracks']['items']:
        names = ''
        for y in x['tracks']["artists"]:
            names = names +'&'+ y['name']

        artistlist_.append(names)    
        songname= x['track']['name']
        idsong= x['track']['id']
        id_list.append(str(idsong))
        songlist_.append(str(songname))

        
dict_['Song'] = songlist_
dict_['Artist'] = artistlist_
dict_['Id'] = id_list   
songs = pd.DataFrame(dict_)
songs.shape


In [ ]:
songs['Id'].nunique()

In [ ]:
songs= songs.drop_duplicates(subset=['Id'])
songs

In [ ]:
Songs_id_list = np.array(songs['Id'])
Songs_id_list

In [ ]:
array_id = np.array_split(Songs_id_list, 21)

In [ ]:
len(array_id[0].tolist())

In [ ]:
len(array_id[0])

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth=acssesToken)

final_data = pd.DataFrame()

for n in list(range(0,21)):
    df2 = sp.audio_features(array_id[n])
    try:
        for i in range(0,99):
            final_data = final_data.append(df2[i], ignore_index=True)
    except IndexError:
            final_data = final_data.append(dict(), ignore_index=True)
    
final_data

In [ ]:
merged = pd.merge(songs,final_data, how = 'left', left_on = 'Id', right_on = 'id')
merged

In [ ]:
merged.to_csv('pop_songs.csv')

In [ ]:
merged['Artist'] = merged['Artist'].str.lstrip("&").str.lower().str.replace(' ','')
merged['Song'] = merged['Song'].str.lower()

In [ ]:
merged['Id'].nunique()